In [5]:
from ultralytics import YOLO
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
import os

In [6]:

# Prüfe GPU-Nutzung
#if torch.cuda.is_available():
#    device = 'cuda'
#    print("GPU-Modus: CUDA")
#else:
device = 'cpu'
print("ACHTUNG: Läuft auf CPU! GPU nicht gefunden.")

# Modell laden (GPU, falls verfügbar)
model = YOLO('yolo11n-pose.pt')
model.to(device)

# Videoquelle (Pfad anpassen, oder 0 für Webcam)
video_path = r'C:\Users\mtwar\Documents\MAS Data Science\CAS Machine Intelligence\Deep Learning\LNW\5_Video\chairlift_vid1.mp4'
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print(f"Video konnte nicht geöffnet werden: {video_path}")
    exit()


ACHTUNG: Läuft auf CPU! GPU nicht gefunden.


In [3]:

# Video-Stream verarbeiten
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Pose-Detektion auf GPU/CPU
    results = model(frame, device=device)

    # Pose-Overlay ins Bild zeichnen
    out_frame = results[0].plot()

    # Im separaten Fenster anzeigen (ESC zum Beenden)
    cv2.imshow("YOLO11n Pose Detection (Live)", out_frame)
    if cv2.waitKey(1) & 0xFF == 27:  # ESC
        break

cap.release()
cv2.destroyAllWindows()



0: 480x640 (no detections), 194.1ms
Speed: 8.6ms preprocess, 194.1ms inference, 6.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 110.3ms
Speed: 4.2ms preprocess, 110.3ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 101.0ms
Speed: 4.9ms preprocess, 101.0ms inference, 7.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 96.6ms
Speed: 4.2ms preprocess, 96.6ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 96.2ms
Speed: 3.0ms preprocess, 96.2ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 98.1ms
Speed: 4.0ms preprocess, 98.1ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 97.2ms
Speed: 4.5ms preprocess, 97.2ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 113.3ms
Speed: 3.0ms preprocess, 113.3ms inference, 2.3ms postprocess per

KeyboardInterrupt: 

In [ ]:
import cv2
import numpy as np

# Laden des OpenPose-Modells
net = cv2.dnn.readNetFromCaffe("pose_deploy.prototxt", "pose_iter_584000.caffemodel")

# Laden des Videos
video_path = r'C:\Users\mtwar\Documents\MAS Data Science\CAS Machine Intelligence\Deep Learning\LNW\5_Video\chairlift_vid1.mp4'
cap = cv2.VideoCapture(video_path)

# Variablen für das Tracking
positions = []
speeds = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Vorbereitung des Frames für das Modell
    frame_width = frame.shape[1]
    frame_height = frame.shape[0]
    inp_blob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (368, 368), (0, 0, 0), swapRB=False, crop=False)
    net.setInput(inp_blob)
    output = net.forward()

    # Extraktion der Positionsdaten
    points = []
    for i in range(15):  # OpenPose COCO Körperpunkte
        prob_map = output[0, i, :, :]
        min_val, prob, min_loc, point = cv2.minMaxLoc(prob_map)
        x = (frame_width * point[0]) / output.shape[3]
        y = (frame_height * point[1]) / output.shape[2]
        if prob > 0.1:  # Schwellenwert für die Erkennung
            points.append((int(x), int(y)))
        else:
            points.append(None)

    # Berechnung des Mittelpunkts der erkannten Punkte
    center_point = None
    valid_points = [p for p in points if p is not None]
    if len(valid_points) > 0:
        center_point = (int(np.mean([p[0] for p in valid_points])), int(np.mean([p[1] for p in valid_points])))
        positions.append(center_point)

    # Berechnung der Geschwindigkeit
    if len(positions) > 1:
        dx = positions[-1][0] - positions[-2][0]
        dy = positions[-1][1] - positions[-2][1]
        distance = np.sqrt(dx**2 + dy**2)
        speed = distance
        speeds.append(speed)
    else:
        speed = 0

    # Zeichnen der Punkte und des Mittelpunkts
    for i, p in enumerate(points):
        if p:
            cv2.circle(frame, p, 5, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
            cv2.putText(frame, "{}".format(i), p, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, lineType=cv2.LINE_AA)

    if center_point:
        cv2.circle(frame, center_point, 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
        # Anzeige der Position und Geschwindigkeit
        cv2.putText(frame, f"Position: {center_point}", (center_point[0] - 100, center_point[1] - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        cv2.putText(frame, f"Speed: {speed:.2f} px/frame", (center_point[0] - 100, center_point[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    cv2.imshow("Frame", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

: 

In [ ]:
import numpy as np
import torch
import seaborn

# Laden des YOLO-Modells
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)  # Beispiel mit YOLOv5

# Laden des Videos
#video_path = r'C:\Users\mtwar\Documents\MAS Data Science\CAS Machine Intelligence\Deep Learning\LNW\5_Video\chairlift_vid1.mp4'
video_path = r'C:\Users\mtwar\Documents\MAS Data Science\CAS Machine Intelligence\Deep Learning\LNW\5_Video\WhatsApp Video 2025-05-12 at 11.45.35.mp4'
cap = cv2.VideoCapture(video_path)

# Variablen für das Tracking
person_positions = {}  # Dictionary zur Speicherung der Positionen jeder Person
next_person_id = 0

# Schwellenwert für die Box-Größe
MIN_BOX_AREA = 500  # Beispielwert, anpassen nach Bedarf


def calculate_iou(box1, box2):
    """Berechnet die Intersection over Union (IoU) zwischen zwei Boxen."""
    x1, y1, x2, y2 = box1
    x1_p, y1_p, x2_p, y2_p = box2

    # Berechnung der Schnittmenge
    xx1 = max(x1, x1_p)
    yy1 = max(y1, y1_p)
    xx2 = min(x2, x2_p)
    yy2 = min(y2, y2_p)

    intersection_area = max(0, xx2 - xx1) * max(0, yy2 - yy1)

    # Berechnung der Vereinigungsfläche
    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x2_p - x1_p) * (y2_p - y1_p)

    union_area = box1_area + box2_area - intersection_area

    iou = intersection_area / union_area if union_area > 0 else 0
    return iou

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Erkennung der Objekte im Frame
    results = model(frame)

    current_frame_boxes = []
    for *xyxy, conf, cls in results.xyxy[0]:
        if int(cls) == 0:  # Klasse 0 ist Person in COCO
            x1, y1, x2, y2 = map(int, xyxy)
            box_width = x2 - x1
            box_height = y2 - y1
            box_area = box_width * box_height

            # Überprüfen, ob die Box größer als der Schwellenwert ist
            if box_area > MIN_BOX_AREA:
                current_frame_boxes.append((x1, y1, x2, y2))

    # Zuordnung der Personen zwischen den Frames
    assigned_ids = set()
    new_person_positions = {}

    for box in current_frame_boxes:
        max_iou = 0
        best_person_id = None

        for person_id, prev_box in person_positions.items():
            iou = calculate_iou(box, prev_box)
            if iou > max_iou and iou > 0.5:  # Schwellenwert für die Zuordnung
                max_iou = iou
                best_person_id = person_id

        if best_person_id is not None:
            assigned_ids.add(best_person_id)
            new_person_positions[best_person_id] = box
        else:
            new_person_positions[next_person_id] = box
            next_person_id += 1

    # Aktualisierung der Positionen und Anzeige der Informationen
    for person_id, box in new_person_positions.items():
        x1, y1, x2, y2 = box
        center_point = ((x1 + x2) // 2, (y1 + y2) // 2)

        # Zeichnen des Rechtecks und des Mittelpunkts
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.circle(frame, center_point, 5, (0, 0, 255), -1)

        # Berechnung der Geschwindigkeit, wenn die Position der Person im vorherigen Frame bekannt ist
        if person_id in person_positions:
            prev_center = ((person_positions[person_id][0] + person_positions[person_id][2]) // 2,
                           (person_positions[person_id][1] + person_positions[person_id][3]) // 2)
            dx = center_point[0] - prev_center[0]
            dy = center_point[1] - prev_center[1]
            distance = np.sqrt(dx**2 + dy**2)
            speed = distance  # Geschwindigkeit in Pixeln pro Frame

            # Anzeige der Position und Geschwindigkeit über dem Kästchen
            cv2.putText(frame, f"Speed: {speed:.2f} px/frame", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        cv2.putText(frame, f"ID: {person_id}", (x1, y1 - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        cv2.putText(frame, f"Position: {center_point}", (x1, y1 - 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    person_positions = new_person_positions

    cv2.imshow("Frame", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

requirements: Ultralytics requirement ['gitpython>=3.1.30'] not found, attempting AutoUpdate...


Using cache found in C:\Users\mtwar/.cache\torch\hub\ultralytics_yolov5_master



requirements: AutoUpdate success  0.7s
WARNING requirements: Restart runtime or rerun command for updates to take effect



YOLOv5  2025-6-2 Python-3.12.10 torch-2.7.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
C:\Users\mtwar/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\mtwar/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\mtwar/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\mtwar/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is

KeyboardInterrupt: 